In [ ]:
import time
import pandas as pd
import straxen
import epix
from epix.simulator.fast_simulator import StraxSimulator

print(f'epix version : {epix.__version__}')

In [ ]:
def monitor_time(prev_time, task):
    t = time.time()
    print(f'It took {(t - prev_time):.4f} sec to {task}')
    return t

In [ ]:
# MC output file as an input for epix/fast_simulator

raw_data_dir='/project2/lgrandi/pkavrigin/'
raw_data_filename='TopUTube_AmBe_100Primaries_WithNVeto.root'

In [ ]:
%rm -r './strax_data'

st = straxen.contexts.xenonnt_simulation()
run_id = '1'
st.register(StraxSimulator)
st.set_config=(dict(nchunk=1, event_rate=5, chunk_size=500,))

# fax config

nt_config=straxen.get_resource('/home/pkavrigin/g4-analysis/private_nt_aux_files/\
sim_files/fax_config_nt_low_field.json', fmt='json')

# epix config

epix_args={'path':raw_data_dir,
           'file_name':raw_data_filename, 
           'debug':True,           
           'entry_start':0,
           'entry_stop':None,
           'cut_by_eventid':False,
           'micro_separation_time':10.0,
           'micro_separation':0.005,
           'tag_cluster_by':'time',
           'max_delay':1e-7,
           'source_rate':-1}

# configuration files; to be moved into the plugin

configuration_files={'s1_relative_lce_map':straxen.pax_file('XENON1T_s1_xyz_lce_true_kr83m_SR1_pax-680_fdc-3d_v0.json'),
                     's2_xy_correction_map':straxen.pax_file('XENON1T_s2_xy_ly_SR1_v2.2.json'),
                     'photon_area_distribution':'XENONnT_spe_distributions_20210305.csv',
                     's1_pattern_map':'XENONnT_s1_xyz_patterns_LCE_corrected_qes_MCva43fa9b_wires.pkl',
                     's2_pattern_map':'XENONnT_s2_xy_patterns_LCE_corrected_qes_MCva43fa9b_wires.pkl',
                     'nv_pmt_qe':'nveto_pmt_qe.json'}

st.config.update(dict(fax_config=nt_config,
                      g4_file=raw_data_dir+raw_data_filename,
                      epix_config=epix_args,
                      xy_resolution=0.5,
                      z_resolution=0.5,
                      configuration_files=configuration_files))

In [ ]:
#st.config
#st.make(run_id, 'events_full_simmed')

sim=StraxSimulator()
sim.config=st.config
sim.setup()

t_start = time.time()
output_sim, output_nveto_hits=sim.compute()
_=monitor_time(t_start, 'run epix and compute.')

In [ ]:
# Simulator output, 'events_full_simmed'

out_df=pd.DataFrame(output_sim)
out_df.sort_values(by='s1_area', ascending=False).head()

In [ ]:
# Simulator output, nVeto hits: each event has a list of pairs [timestamp, pmt_channel]; each pair corresponds to a generated photoelectron

output_nveto_df=pd.DataFrame(output_nveto_hits)
output_nveto_df[output_nveto_df.hits.apply(lambda x: len(x) > 0)].head()